# Data Wrangling 2: Data Reformatting

In the previous lesson, we discuss ways of cleaning the data through filtering the data and correcting data types. Continuing the discussion on Data Wrangling, we will talk about how to reformat your data frame through pivoting and melting data frames, multi-indexed data frames, and joining additional data dataframes. 

## Restructuring Data
Depending on where you obtain your data, you might receive it in one of two formats. These are most commonly referred to as **wide data** or **long data**. The wide data format is one where each column contains a single type of data. This is what most people commonly use in their spreadsheet data. For example, you might store information like business name, business address, phone number, and contact name in seperate columns. Each row here would then represent one business. Let's examine in an example some temperature data stored in of a wide format. 

In [ ]:
import pandas as pd
wide_data = pd.read_csv("https://raw.githubusercontent.com/stefmolin/Hands-On-Data-Analysis-with-Pandas/master/ch_03/data/wide_data.csv")
wide_data.head()

In the above dataframe, notice that each type of measured value is a unique type of thing. In this case, they are all numbers, but `TMAX` *only* reports the maximum temperature for the day. This format is great for presentation and analysis. When running the method `df.describe()`, we obtain sensible information on each row.

In [ ]:
wide_data.describe()

Wide data is very commonly used in databases. Due to the fact that each column has a unique type of value, relational databases can host a suite of tables that all have information that is linked together through keys. This key allows us to join only the necessary tables together to get information that we need without much excess.

One drawback to this design is that it can be fairly inflexible. Using the temperature data above, imagine an experiment where we needed to add a new measurement to our experiment, for example, air pressure, cloud coverage, or rainfall. As far as the experiment is concerned, all you need to do is add a new sensor. However, if the data is being stored in a database, you will need to perform a schema change, account for the previous rows of missing information (especially if it's a column that cannot be null), and need to update your documentation if you are sharing the data with anyone. While, not a terrible ask, it can be resource intensive, and may cause extra complication to a potentailly complex web of tables in your database. 

In contrast, we could store the same data in a long format, we will have 3 important columns, or sets of columns: 
- a column (or set of columns) to identify the observation
- variable name or key
- the value being measured 

With all 3 of these bits of information, we possess the exact same amount of information with the wide format, however formatted differntly, We will have (potentially) significantly fewer columns, however, with a much higher number of rows. Below is the exact same temperature data from above formatted in this long style. 

In [ ]:
long_data = pd.read_csv('https://raw.githubusercontent.com/stefmolin/Hands-On-Data-Analysis-with-Pandas/master/ch_03/data/long_data.csv')[['date', 'datatype', 'value']]
long_data.head(15)

If we carefully compare the above table with the wide table that we examined above, we will see that all the same data is present. In this table, our key column is called `datatype` and the measured value is under `value`. We also will notice that each value of the data is repeated 3 times, and that the values in `datatype` are the names of the columns in the wide table. In this table, we have restructured the data such that all the columns of the previous table are compressed into these two columns. This format would be very confusing to show another person, but it does remove the need to create new columns. If you start recording a new type of measurement, simply create another key/`datatype` value and start recording the data. This format is often found with dealing with APIs, but would also make for terrible relational database design. Additionally, any attempt at producing summary statistics on this table would be nonsensical.

Note that each format has its benefits and reasons for use. Therefore, it is important to know how to switch between the two as needed. Let's start with how to change from long to wide. 

### Pivoting DataFrames
Converting from a long format to a wide format creates what is often called a **pivot table**. Pandas has a method built into the `DataFrame` class to perform exactly this conversion called `pivot`. This method requires columns to be labled as the index (observation identifer), the column containing the future column names (the variable names or keys) and the column containing the measurements or values. Pivoting the long formatted temperature data creates the following dataframe.

In [ ]:
pivot_df = long_data.pivot(
    index='date', 
    columns='datatype',
    values='value'
)
pivot_df.head()

Now we have created a new table that looks like the first wide table that we read in. Now, we could join this table with other datasets, or extract summary statistics.

One additional feature of Pandas is to set up your data with a **hierarchical index**. If you recall from last week, this `value` columns was actual the temperature in Celcius. So, let's rename it as such and include the temperature in Fahrenheit. 

In [ ]:
long_data.rename(columns={"value":"temp_C"}, inplace=True)
long_data["temp_F"] = (long_data.temp_C * 9/5) + 32

In [ ]:
long_data.head()

When we pass in multiple columns to the `values` argument, we end up with the following dataframe.

In [ ]:
pivot_df = long_data.pivot(
    index='date', 
    columns='datatype',
    values=['temp_C', 'temp_F']
)
pivot_df.head()

We now have two of each column name, one under Celcius, and one under Fahrenheit. Under this structure, Each higher column name actually returns a dataframe.

In [ ]:
pivot_df['temp_C'].head()

Therefore, in order to reference any single column, we need to provide (what looks like) two column names. 

In [ ]:
pivot_df['temp_C']['TMAX'].head()

In [ ]:
pivot_df.temp_C.TMAX.head()d

### Multi-Index
Pandas also allows for option to provide multiple indices for a particular row, just like we have provided multiple indices for a column (remember, column headers are actually treated as index objects). To utilize this, we call the familiar `set_index` object, and pass in multiple columns, in the order that they should be grouped. 

Let's examine setting the index by `date`, then by `datatype`.

In [ ]:
multi_index_df = long_data.set_index(['date','datatype'])
multi_index_df.head(6)

Here we have grouped the rows together based on the date, and have the `datatype` column as a second index. However, this is still our long format. Additionally, the `pivot` method expects a dataframe with only one index. To achieve a similar table as `pivot`, we can use the `unstack` method, which removes all layers of your multi-index and turns the into columns. Essentially, picture as if we have stacked the values on top of each other, and we are now unstacking them, and laying them all out into a single row. 

In [ ]:
unstacked_df = multi_index_df.unstack()
unstacked_df.head()

Alternatively, we could have reversed the order of the indicies to create a different grouping.  

In [ ]:
type_indexed_df = long_data.sort_values(["datatype", "date"]).set_index(["datatype", "date"])
type_indexed_df.head(10)

Here, we have created a different ordering where all the data types are grouped together, with the dates (eventually) repeating over the length of the dataframe (Trying to display all 90 rows will hide the rows in the middle). Just like we could interact just part of this dataframe with the higher order column reference, we can reference just a part of the dataframe with the higher order index to get just a part of our dataframe. Here we grab all the information for a specific datatype (where the date is the higher order index). 

In [ ]:
type_indexed_df.loc['TMAX']

A similar action could be achieved with if we had reordered to have the date first, but, would be best achieved if we had set the column to a date first.   

### Melting DataFrames
Melting is the inverse action of a pivot. A pivot takes rows and combines them into a single column, melting a dataframe takes the columns of a dataframe and converts them into rows. By calling the method `melt`, we can pass in the arguments: 
- `id_vars`: which columns uniquely identify a row (if requiring multiple columns, this is called a composite key)
- `value_vars`: which columns contain values
- `var_name`: what to name the column with the column names (default is `variable`)
- `value_name`: what to name the column with the values of those columns (default is `value`)

In [ ]:
melted_df = wide_data.melt(
    id_vars='date',
    value_vars=['TMAX', 'TMIN', 'TOBS'],
    value_name='temp_C',
    var_name='measurement'
)
melted_df.head()

Alternatively, we have `stack`, which naturally is the inverse of `unstack`. One significant difference is that this will create a series, instead of a dataframe (at least for our current dataframes). This means that we loose the column names, and there are some differences to take into consideration while using this slightly altered format. 

In [ ]:
stacked_series = wide_data.set_index('date').stack()
stacked_series.head()

Or, we could also convert it to a dataframe by running `to_frame` and passing a column name. 

In [ ]:
stacked_df = stacked_series.to_frame('values')
stacked_df.head()

We can add a name to the second index, now that it is actually a multi-index by calling `rename` on the `index` object for the `dataframe`. 

In [ ]:
stacked_df.index.rename(['date', 'datatype'], inplace=True)
stacked_df.head()